In [1]:
#include <iostream>
#include <deque>
#include "../src/SinglyLinkedNode.h"
using namespace std;

# Iterators

## Accessing the ADT Entries

- There are three mechanisms for accessing entries in the ADT, e.g. deque:
    - Two random access member functions
        - An overloaded indexing operator		`ideque[10]`
        - The `at()` member function; and		`ideque.at( 10 )`
    - The iterator design pattern

- The difference between indexing and using the function `at( int )` is that the second will throw an `out_of_range` exception if it accesses an entry outside the range of the container


## Stepping Through Lists

You should be familiar with this technique of stepping through a list:

```cpp
ArrayList<int> list;
for ( int i = 0; i < list.size(); ++i ) { cout << list.at( i ); }
```

or

```cpp
SinglyLinkedList<int> list;
for ( SinglyLinkedNode<int> *ptr = list.head(); ptr != 0; ptr = ptr->next() ) {
    cout << ptr->value();
}
```

- There are serious problems with this approach:
    - It exposes the underlying structure
    - It is impossible to change the implementation once users have access to the structure
    - The implementation will change from class to class
        - `SinglyLinkedList` requires `SinglyLinkedNode`
        - `DoublyLinkedList` requires `DoublyLinkedNode`
    - An array-based data structure does not have a direct analogy to the concept of either `head()` or `next()`

- More critically, what happens with the following code?    
```cpp
SinglyLinkedNode<int> *ptr = list.front();
list.pop_front();
cout << ptr->value() << endl; /// ???
```

- Or how about ...
```cpp
ArrayList<int> list;
for ( int i = 0; i < 10; ++i ) { list.push_front( i ); }
delete list.front();        // ?!	
```

## Iterators

- Above list implementations expose the underlying data structure for evaluation purposes
    - This is, however, not good programming practice

- The C++ STL uses the concept of an iterator to solve this problem
    - The iterator is not unique to C++
    - It is an industry recognized approach to solving a particular problem
    - Such a solution is called a *design pattern*
        - Formalized in [Design Patterns by E.Gamma et al.](https://en.wikipedia.org/wiki/Design_Patterns)

## Standard Template Library Iterators

- Associated with each STL container class is an *nested class* termed an **iterator**
    - `deque<int> ideque;`
    - `deque<int>::iterator itr`;

- The iterator "refers" to one position within the deque
    - It is similar a pointer but is **independent** of implementation

## Analogy

- Consider a filing system with an administrative assistant
- Your concern is not how reports are filed (so long as it's efficient), it is only necessary that you can give directions to the assistant
- You can request that your assistant
    - Starts with either the first or last file
    - You can request to see the file the assistant is currently holding
    - You can modify the file the assistant is currently holding
    - You can request that the assistant either:
    - Go to the next file, or
    - Go to the previous file

## C++ Iterators

- In C++, iterators **overloads** a number of operators:
    - The unary `*` operator returns a reference to the value stored at the location pointed to by the iterator
    - The operator `++` updates the iterator to point to the next position
    - The operator `--` updates the iterator to point to the previous location

- Note: these look like, but are not, **pointers**...

In [2]:
deque<int> ideque;
ideque.push_front( 5 ); ideque.push_back( 4 );
ideque.push_front( 3 ); ideque.push_back( 6 );
// the deque is now 3 5 4 6
ideque

{ 3, 5, 4, 6 }

- We request an iterator on a specific instance of a deque by calling the member function `begin()`
![](../img/iter1.png)

In [3]:
deque<int>::iterator itr = ideque.begin();

- We access the value by calling `*itr`
![](../img/iter2.png)

In [4]:
{
    cout << *itr;
}

3

Similarly, we can modify the value by assigning to `*itr`:
![](../img/iter3.png)

In [5]:
{
    cout << *itr << " == " << ideque.front() << endl;
    *itr = 11; // assignment operator
    cout << *itr << " == " << ideque.front() << endl;
}

3 == 3
11 == 11


- We update the iterator to refer to the next value by calling increment `++itr`

![](../img/iter3.png)

```cpp
++itr;
```

![](../img/iter4.png)

In [6]:
{
    cout << *itr << " ";
    ++itr; // prefix increment operator
    cout << *itr;
}

11 5

- The iterators returned by `begin()` and `end()` refer to
    - The **first** position (head) in the container, and
    - The position **after the last** value in the container.
![](../img/iter5.png)

- The reverse iterators returned by `rbegin()` and `rend()` refer to:
    - the **last** position (tail) in the container, and
    - the position **before the first** location in the container.
![](../img/iter5.png)

- If a container is empty then the beginning and ending iterators are **equal**

In [7]:
{
    deque<int> cntr;

    cout << (  cntr.begin() == cntr.end()  ) << " "
         << ( cntr.rbegin() == cntr.rend() ) << endl;
}

1 1


- Because we can have multiple iterators referring to values within the same container, it makes sense that we can use the comparison operator `==` and `!=`

- The folloing code gives some suggestion as to why `end()` refers to the position **after the last** location in the container

In [8]:
ideque

{ 11, 5, 4, 6 }

In [9]:
for ( int i = 0; i != ideque.size(); ++i ) {
    cout << ideque[i] << endl;
}

11
5
4
6


In [10]:
for ( deque<int>::iterator itr = ideque.begin(); itr != ideque.end(); ++itr ) {
    cout << *itr << endl;
}

11
5
4
6


- Modifying something beyond the last location of the container results in **undefined behaviour**

- Do not use `end()` iterator for updates

In [11]:
deque<int>::iterator itr = ideque.end();
*itr = 3; // wrong
ideque

{ 11, 5, 4, 6 }

- You should use the correct member functions

In [12]:
ideque.push_back( 3 ); // right
ideque

{ 11, 5, 4, 6, 3 }

In [13]:
deque<int>::iterator itr = ideque.begin();
cout << *itr << endl;
++itr;
cout << *itr << endl;

while ( itr != ideque.end() ) {
    cout << *itr << " ";
    ++itr;
}

11
5
5 4 6 3 

## Why Iterators?

- Now that you understand what an iterator does, lets examine why this is standard software-engineering solution; they
    - Do not expose the underlying structure
    - Require $\Theta(1)$ additional memory
    - Provide a common interface which can be used regardless of whether or not it's a vector, a deque, or any other data structure
    - Do not change, even if the underlying implementation does

## Iterator Classes

- These are not real C++ classes, but simply [categories](https://users.cs.northwestern.edu/~riesbeck/programming/c++/stl-iterators.html) of **kind of iterators**.
    - Each category specifies the operations the iterator supports, see [Iterator library](https://en.cppreference.com/w/cpp/iterator).

- For example, some iterators support incrementing but not decrementing, some support dereferencing for getting data but not for storing data, etc.

- Each STL container defines what class of iterators it can return. 
    - See **Member types** section for [list](https://en.cppreference.com/w/cpp/container/list) or [deque](https://en.cppreference.com/w/cpp/container/deque)
- Each algorithm specifies what class of iterators it requires.    
    - E.g. some functions from `<algorithm>` library: [sort](https://en.cppreference.com/w/cpp/algorithm/sort) or [partition](https://en.cppreference.com/w/cpp/algorithm/partition)

## InputIterator

- `InputIterator` is a useful but limited class of iterators. If `iter` is an `InputIterator`, you can use:

    - `++iter` and `iter++` to increment it, i.e., advance the pointer to the next element
    - `*iter` to dereference as `rvalue`, i.e., get the element value pointed to
    - `==` and `!=` to compare it another iterator (typically the "end" iterator)
    - its value type does not need to be assignable, `t = u` is not required

## OutputIterator

- `OutputIterator` is another limited class of iterators. If `iter` is an `OutputIterator`, you can use:

    - `++iter` and `iter++` to increment it, i.e., advance the pointer to the next element
    - Can be dereferenced as an `lvalue`, to store data in the location pointed to, `*iter = ...`

## `ForwardIterator`

- `ForwardIterator` combines `InputIterator` and `OutputIterator`.
    - You can use them to read and write to a container.
    - They also support saving and reusing

In [14]:
vector<int> v;
vector<int>::iterator iter, iterSaved;
v.push_back(1); v.push_back(2); v.push_back(3);

In [15]:
iter = v.begin();
iterSaved = iter;
iter++;
cout << "Previous element is " << (*iterSaved) << endl;
cout << "Current element is " << (*iter) << endl; 
*iter = 10;
cout << "New current element is " << (*iter) << endl; 

Previous element is 1
Current element is 2
New current element is 10


In [16]:
for (iter = v.begin(); iter != v.end(); iter++)
  cout << (*iter) << " ";

1 10 3 

## BidirectionalIterator

If `iter` is a `BidirectionalIterator`, you can use:

- all `ForwardIterator` operations
- `--iter` and `iter--` to decrement it, i.e., advance the pointer to the previous element

In [17]:
iter = v.begin();
iter++;
cout << "Current element is " << *iter << endl; 
iter--;
cout << "Previous element is " << *iter << endl;

Current element is 10
Previous element is 1


## RandomAccessIterator

- If `iter1` and `iter2` are `RandomAccessIterator`'s, you can use:

- all `BidirectionalIterator` operations
- standard pointer arithmetic, i.e., `iter + n`, `iter - n`, `iter += n`, `iter -= n`, and `iter1 - iter2` (but not `iter1 + iter2`)
- all comparisons, i.e., `iter1 > iter2`, `iter1 < iter2`, `iter1 >= iter2`, and `iter1 <= iter2`

In [18]:
iter = v.begin();
cout << "Current element is " << *iter << endl; 
cout << "Next element is " << (*(iter+1)) << endl;
cout << "After next element is " << (*(iter+2)) << endl;

Current element is 1
Next element is 10
After next element is 3


- Since `BidirectionalIterator`'s support `++` and `--`, don't they support these operations too?
    - The answer is that `RandomAccessIterator`'s support these operations in constant time.
        - That is, you can jump `N` elements in the same time it takes to jump 1 element.
    - So an STL `list` container can return a `BidirectionalIterator`, but not a `RandomAccessIterator`.
    - `vector` and `deque` can return `RandomAccessIterator`.

In [19]:
typedef SinglyLinkedNode<int> SLN;

class ListWithIterator {
    SLN* head;

public:
    class iterator {
        friend ListWithIterator; // for access to the parent list class memebers
    public:
        iterator(): curr{nullptr}{} // empty default constructor
        int operator* () const { return curr->value(); }
        iterator operator++ () { // prefix increment
            curr = curr->next(); return *this;
        }        
        bool operator== ( const iterator & rhs ) const { return curr == rhs.curr; }
    protected:
        SLN *curr; // current list node
        iterator(SLN* node): curr{node}{} // iterator constructor        
    };
    
    ListWithIterator(SLN* h = nullptr) : head{h} {} // list constructor
    ~ListWithIterator() { while(!empty()) pop_front(); } // list destructor
    bool empty() { return head == nullptr; } 
    SLN* front() { return head; }
    void pop_front() { auto tmp = head; head = head->next(); delete tmp; };
    
    iterator begin() const { return iterator( head ); }
    iterator end() const { return iterator(); }
};

In [20]:
{
    SLN* h = new SLN(1, new SLN(2, new SLN(3, nullptr)));
    ListWithIterator lst{h};
    
    cout << "Direct list access: ";
    for( auto ptr = lst.front(); ptr != nullptr; ptr=ptr->next())
        cout << ptr->value() << " ";
    
    cout << "\n\nFrom iterator: ";
    for( ListWithIterator::iterator iter = lst.begin(); !(iter == lst.end()); ++iter)
        cout << *iter << " " ;
}

Direct list access: 1 2 3 

From iterator: 1 2 3 